## MULTIWAY CUT LP relaxation
### Graph based on viewers rating movies
Here we solve the relaxed LP to find the multiway cut in a graph. The goal of the minimum multiway Cut is to find a set of edges $E_2 \subseteq E$ with minimum weight such that removing $E_2$ from $G$ separates all terminals nodes preselected.

In [ ]:
%matplotlib inline
import networkx as nx
import numpy as np
import picos as pic
import pylab

from algo import build_laplacian, solve_multicut, assignment_solution_lp
from data_proc import build_a, build_graph, build_user_to_movies

In [ ]:
np.set_printoptions(precision=2)

## Fake data to illustrate the integral solution
Easy multiway cut solution with 3 terminal nodes.

In [ ]:
W = np.array([[0,10,0,0,0,0,1,15],
              [10,0,1,0,0,0,0,0],
              [0,1,0,10,0,0,0,0],
              [0,0,10,0,1,0,0,0],
              [0,0,0,1,0,10,0,0],
              [0,0,0,0,10,0,10,0],
              [1,0,0,0,0,10,0,1],
              [15,0,0,0,0,0,1,0],
             ])
T = [0, 2, 5]

In [ ]:
D = solve_multicut(W, T)

In [ ]:
assignment_D = assignment_solution_lp(D, T, threshold=0.1)

In [ ]:
#display the graph
G = nx.from_numpy_matrix(W)
fig=pylab.figure(figsize=(11,8))

colors = ['b', 'g', 'r']
cluster_colors = dict()
for cluster_number, cluster in enumerate(assignment_D):
    cluster_colors[cluster] = colors[cluster_number]


node_colors=['w']*W.shape[0]
for cluster in assignment_D:
    nodes = assignment_D[cluster]
    for node in nodes:
        node_colors[node] = cluster_colors[cluster]

pos=nx.spring_layout(G)
#edges
nx.draw_networkx(G,pos,
                edgelist=[e for e in G.edges()],
                node_color=node_colors)


labels={e:'{0}'.format(W[e]) for e in G.edges()}
#flow label
nx.draw_networkx_edge_labels(G, pos,
                        edge_labels=labels)

#hide axis
fig.gca().axes.get_xaxis().set_ticks([])
fig.gca().axes.get_yaxis().set_ticks([])

pylab.show()

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 5
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W.copy()
W_gamma2[mask] = r[mask]
D = solve_multicut(W_gamma2,T)
print(D)
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

## Simulated data
Simulating a bipartite graph with viewers giving grades (between -2.5 and 2.5) to movies.

In [ ]:
G=nx.Graph()

G.add_edge(1,5,weight=1)
G.add_edge(1,6,weight=1)
G.add_edge(2,5,weight=1)
G.add_edge(2,6,weight=1)
G.add_edge(2,7,weight=-1)
G.add_edge(2,8,weight=-1)
G.add_edge(3,7,weight=1)
G.add_edge(3,8,weight=1)
G.add_edge(3,10,weight=2.5)
G.add_edge(3,9,weight=0.5)
G.add_edge(4,7,weight=-2)
G.add_edge(4,9,weight=2.5)
G.add_edge(4,10,weight=0.5)
W = nx.to_numpy_matrix(G)
T = [5,7,9]
D = solve_multicut(W,T)
print(np.reshape(D, W.shape))
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

Seeing that the graph is not 4-stable, we try to distort it (it still has a meaning when considering movie ratings)

In [ ]:
W_sign = np.sign(W)
W_abs = np.absolute(W)
W_exp_abs = np.exp(W_abs)
W_distorted = W_sign * W_exp_abs
D = solve_multicut(W_distorted,T)
print(np.reshape(D, W_distorted.shape))
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 2
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W.copy()
W_gamma2[mask] = r[mask]
L = build_laplacian(W_gamma2)
D = solve_multicut(L,T)
print(D)
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

## Real data from themoviedb.org
Extracting  a sub graph of the entire database

In [ ]:
# building the summary dictionary
summary_dictionary = build_user_to_movies('movielens.tsv')
# unpacking of the dictionary
users_to_movies = summary_dictionary['users_to_movies']
n_users = summary_dictionary['n_users']
k_users = 6
n_movies = summary_dictionary['n_movies']
k_movies = 8
# building the rating matrix
a = build_a(n_users, k_users, n_movies, k_movies, users_to_movies)
# building the adjacency matrix
W = build_graph(k_users, k_movies, a)
print(W)

In [ ]:
# W = np.load('good_real_matrix.npy')

In [ ]:
T = [0,2, 4]
D = solve_multicut(W,T)
D_array=np.array(D)
assignment_D = assignment_solution_lp(D_array, T, threshold=0.00001)

In [ ]:
#display the graph
G = nx.from_numpy_matrix(W)
fig=pylab.figure(figsize=(11,8))

colors = ['b', 'g', 'r']
cluster_colors = dict()
for cluster_number, cluster in enumerate(assignment_D):
    cluster_colors[cluster] = colors[cluster_number]


node_colors=['w']*W.shape[0]
for cluster in assignment_D:
    nodes = assignment_D[cluster]
    for node in nodes:
        node_colors[node] = cluster_colors[cluster]

pos=nx.spring_layout(G)
#edges
nx.draw_networkx(G,pos,
                edgelist=[e for e in G.edges()],
                node_color=node_colors)


labels={e:'{0}'.format(W[e]) for e in G.edges()}
#flow label
nx.draw_networkx_edge_labels(G, pos,
                        edge_labels=labels)

#hide axis
fig.gca().axes.get_xaxis().set_ticks([])
fig.gca().axes.get_yaxis().set_ticks([])

pylab.show()

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 2
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W.copy()
W_gamma2[mask] = r[mask]
L = build_laplacian(W_gamma2)
D = solve_multicut(L,T)
print(D)
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)